# Applying Machine Learning and Deep Learning to identify home appliances consuming excess power

## Copyright (c) 2018, Faststream Technologies
## Author: Sudhanva Narayana

In [1]:
# Home Appliances Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Import Dataset
df_home_data_test = pd.read_csv('../../../data/trial_2/home_data_predict_dt.csv')
df_home_priority = pd.read_csv('../../../data/trial_2/home_priority.csv')
date_time_group = pd.read_csv('../../../data/trial_2/date_time_group.csv')

In [3]:
df_home_data_test.head()

,device,room,weather_type,date,from_time,to_time,time,no_of_people,time_stayed_mins,power
0,AC,100,cold,2018-01-01,04:00:00,05:00:00,early morning,6,24,1880.0
1,AC,100,low hot,2018-01-01,05:00:00,06:00:00,early morning,0,0,1880.0
2,AC,100,low hot,2018-01-01,07:00:00,08:00:00,morning,13,38,1304.0
3,AC,100,cold,2018-01-01,09:00:00,10:00:00,morning,14,7,1766.0
4,AC,100,low hot,2018-01-01,09:00:00,10:00:00,morning,10,26,1702.0


In [4]:
df_home_priority.head()

,device,weather_type,time,priority,order
0,AC,low cold,early morning,8,52
1,AC,low cold,morning,8,49
2,AC,low cold,afternoon,8,48
3,AC,low cold,evening,8,50
4,AC,low cold,night,8,51


In [5]:
date_time_group.head()

,date,from_time,power
0,2018-01-01,00:00:00,10372
1,2018-01-01,01:00:00,9393
2,2018-01-01,02:00:00,10013
3,2018-01-01,03:00:00,10171
4,2018-01-01,04:00:00,10650


In [6]:
total_power_consumption = df_home_data_test.groupby(['date', 'from_time']).sum()['power'].reset_index()

In [7]:
messages = []
actions = []
savings = []

In [8]:
for main_index, main_row in df_home_data_test.iterrows():
    sample_date = main_row['date']
    sample_time = main_row['from_time']

    sample_df = df_home_data_test[(df_home_data_test['date'] == sample_date) & (df_home_data_test['from_time'] == sample_time)]
    sample_total_power = sample_df['power'].sum()

    power = date_time_group[(date_time_group['date'] == sample_date) & (date_time_group['from_time'] == sample_time)]
    condition = sample_total_power > power['power'][power['power'].index[0]]

    priorities = []

    for index, row in sample_df.iterrows():
    #     print(row['device'], row['weather_type'], row['time'])
        priorities.append(df_home_priority[(df_home_priority['device'] == row['device']) 
                               & (df_home_priority['weather_type'] == row['weather_type']) 
                               & (df_home_priority['time'] == row['time'])].values[0][4])

    priorities = pd.Series(priorities)

    sample_df = sample_df.assign(priorities=priorities.values)
    sample_df = sample_df.sort_values(['priorities', 'no_of_people'], ascending=[True,False])
    old_df = sample_df.copy()
    sample_after_power = 10000000

    while sample_total_power < sample_after_power:
        no_of_people_index = sample_df.iloc[[-2]]['no_of_people'].index[0]

        last_one_room = sample_df.iloc[[-2]]['room'].values[0]
        last_no_of_people = sample_df.iloc[[-1]]['no_of_people'].values[0]
        last_one_no_of_people = sample_df.iloc[[-2]]['no_of_people'].values[0]
        last_room = sample_df.iloc[[-1]]['room'].values[0]
        last_device = sample_df.iloc[[-1]]['device'].values[0]
        
        sample_df.at[no_of_people_index, 'no_of_people'] = last_no_of_people + last_one_no_of_people 
        sample_df_drop = sample_df.drop(sample_df.tail(1).index) 
        sample_after_power = sample_total_power - sample_df_drop['power'].sum()

        action = 'Turn off ' + last_device + ' in room ' + str(last_room)
        
        if last_no_of_people != 0:
            message = 'Moving ' + str(last_no_of_people) + ' people from room ' + str(last_room) + ' to room ' + str(last_one_room) + ' saves ' + str(sample_after_power) + ' of' + ' electricity, ' + 'power consumption will reduce from ' +  str(sample_total_power) + ' to ' + str(sample_total_power - sample_after_power)
        else:
            message = 'None'
            
        messages.append(message)
        actions.append(action)
        savings.append(sample_after_power)

In [9]:
sample_df

,device,room,weather_type,date,from_time,to_time,time,no_of_people,time_stayed_mins,power,priorities
4923,Television,118,hot,2018-01-14,23:00:00,00:00:00,midnight,10,10,294.0,229
5207,Television,119,hot,2018-01-14,23:00:00,00:00:00,midnight,6,31,284.0,229
2001,Refridgerator,107,low cold,2018-01-14,23:00:00,00:00:00,midnight,7,27,145.0,241
3061,Refridgerator,111,very cold,2018-01-14,23:00:00,00:00:00,midnight,13,56,264.0,253
3823,Refridgerator,114,very cold,2018-01-14,23:00:00,00:00:00,midnight,0,0,143.0,253
2279,Refridgerator,108,hot,2018-01-14,23:00:00,00:00:00,midnight,13,43,166.0,265
954,Refridgerator,103,very hot,2018-01-14,23:00:00,00:00:00,midnight,9,47,153.0,271


In [10]:
df_home_data_test['message'] = messages
df_home_data_test['action'] = actions
df_home_data_test['savings'] = savings

df_home_data_test.to_csv('../../../data/trial_2/final_output_dt.csv', index=False)